In [1]:
import pandas as pd

data = pd.read_csv("listings_new_york_2024.csv")

data.sample(1)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
36039,1134900809771751391,Stylish 3 bedrooms in Brooklyn,503294089,Nir,Brooklyn,Crown Heights,40.672919,-73.931492,Entire home/apt,NaN,30,0,NaN,NaN,1,269,0,NaN


In [12]:
rows, columns = data.shape
print("rows = ", rows)
print("columns = ", columns)

rows =  37765
columns =  18


In [16]:
data.dtypes

<bound method NDFrame.describe of                         id                                               name  \
0                     2595                              Skylit Midtown Castle   
1                     5136    Spacious Family Friendly Duplex w/ Patio + Yard   
2                     6848                   Only 2 stops to Manhattan studio   
3                     6872  Uptown Sanctuary w/ Private Bath (Month to Month)   
4                     6990                            UES Beautiful Blue Room   
...                    ...                                                ...   
37760  1192946306020922085                         Private Studio in Brooklyn   
37761  1193005542763264121                   Skyline Views from MSG Penthouse   
37762  1193016130087171760  Stunning designer Chelsea studio on the best b...   
37763  1193068036013405081                         Sunny & Spacious in Queens   
37764  1193086058992851452                        Bright 2 Bedroom in Astor